In [1]:
from pathlib import Path
import os

RESULTS_ROOT = Path("/mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC")

# Put the experiment folder names you want to clean here
EXPERIMENTS_TO_CLEAN = [
    "exp60",
    "exp61",
    "exp62",
    "exp63_uniform",
    "exp64_uniform_corr",
    "exp65_partial_l1",
    "exp66_mm",     

]

DRY_RUN = True  # <-- set False to actually delete


def bytes_to_gb(n: int) -> float:
    return n / (1024**3)


def find_pth_in_pruned(exp_dir: Path) -> list[Path]:
    """
    Find all .pth files under exp_dir/pruned/** only.
    This automatically excludes baseline because baseline is not inside pruned/.
    """
    pruned_dir = exp_dir / "pruned"
    if not pruned_dir.exists():
        return []
    return sorted(pruned_dir.rglob("*.pth"))


def summarize_files(files: list[Path], top_k: int = 15):
    sizes = [(p, p.stat().st_size) for p in files if p.exists()]
    total_bytes = sum(s for _, s in sizes)
    sizes_sorted = sorted(sizes, key=lambda x: x[1], reverse=True)
    print(f"Found {len(files)} .pth files")
    print(f"Total size: {bytes_to_gb(total_bytes):.3f} GB")
    if sizes_sorted:
        print(f"\nTop {min(top_k, len(sizes_sorted))} largest:")
        for p, s in sizes_sorted[:top_k]:
            print(f"  {bytes_to_gb(s):.3f} GB  |  {p}")
    return total_bytes


grand_total = 0
all_targets: list[Path] = []

print(f"Results root: {RESULTS_ROOT}")
print(f"DRY_RUN={DRY_RUN}\n")

for exp_name in EXPERIMENTS_TO_CLEAN:
    exp_dir = RESULTS_ROOT / exp_name
    if not exp_dir.exists():
        print(f"⚠️  {exp_name}: folder not found at {exp_dir}")
        continue

    targets = find_pth_in_pruned(exp_dir)
    all_targets.extend(targets)

    print(f"\n=== {exp_name} ===")
    total_bytes = summarize_files(targets, top_k=10)
    grand_total += total_bytes

print("\n======================")
print(f"TOTAL reclaimable (estimated): {bytes_to_gb(grand_total):.3f} GB")
print(f"TOTAL files: {len(all_targets)}")
print("======================\n")

if not DRY_RUN:
    deleted = 0
    failed = 0
    for p in all_targets:
        try:
            p.unlink()
            deleted += 1
        except Exception as e:
            failed += 1
            print(f"❌ Failed to delete {p}: {e}")

    print(f"\n✅ Deleted {deleted} files")
    if failed:
        print(f"⚠️ Failed deletions: {failed}")
else:
    print("Dry run only — no files deleted. Set DRY_RUN=False to delete.")


Results root: /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC
DRY_RUN=True


=== exp60 ===
Found 23 .pth files
Total size: 0.196 GB

Top 10 largest:
  0.014 GB  |  /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC/exp60/pruned/l1_norm_0_0_0_99_99_99_99_99_0_0_0/pruned_model/pruned_model.pth
  0.014 GB  |  /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC/exp60/pruned/l1_norm_0_0_0_99_99_99_99_99_0_0_0_random/pruned_model/pruned_model.pth
  0.014 GB  |  /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC/exp60/pruned/l1_norm_0_0_0_99_99_99_99_99_0_0_0_rewind/pruned_model/pruned_model.pth
  0.014 GB  |  /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC/exp60/pruned/l1_norm_0_0_0_99_99_99_99_99_0_0_0_random/retraining_pruned/final_model.pth
  0.009 GB  |  /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC/exp60/pruned/l1_norm_0_0_0_99_99_99_99_99_50_0_0/pruned_model/pruned_model.pth
  0.009 GB  |  /mnt/hdd/ttoxopeus/basic_UNet/results/UNet_ACDC/exp60/pruned/l1_norm_0_0_0_99_99_99_99_99_50_0_0/retraini